In [1]:
import pandas as pd
import numpy as np
import os
import sklearn

import pickle

In [ ]:
original_df = pd.read_csv('carrefour_sales.csv')


# Read Data & preprocessing¶
columns: id, order_date, product, sales_price, quantity, department, store, city, district, user_id, sex, age_group add total_price, order_date_notime columns drop quantity <0

## 拆分資料

In [6]:
original_df.columns=["id","order_date","product","sales_price","quantity","department","store","city","district","user_id","sex","age_group"]
original_df['total_price'] = original_df['sales_price']*original_df['quantity']
original_df['order_date_notime'] = pd.to_datetime(original_df['order_date']).dt.floor('d')
original_df['order_month'] = original_df['order_date_notime'].apply(lambda x: x.month)
original_df['order_year'] = original_df['order_date_notime'].apply(lambda x: x.year)
original_df['dt'] = (original_df['order_year']-2020)*12 + original_df['order_month']

# label encoding：product
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
original_df = original_df.rename(columns={'product': 'product_oroginal', 'user_id': 'user_id_original'})
original_df['product'] = labelencoder.fit_transform(original_df['product_oroginal'])
original_df['user_id'] = labelencoder.fit_transform(original_df['user_id_original'])
original_df.head()

,id,order_date,product_oroginal,sales_price,quantity,department,store,city,district,user_id_original,sex,age_group,total_price,order_date_notime,order_month,order_year,dt,product,user_id
0,10eed4a6-5b6b-40d9-84fe-e7152666d2d5,2021-07-09 17:32:43,99d32476-f1eb-4af8-a7d9-28ebd8e1ce12,85,3,7ed47ad3-61a2-423c-937b-c23bf4ad05f3,38177bc5-93b3-4b7a-ad27-d6aa162ccc9a,0ced232a-ba28-4238-8411-264c228aa751,de4598ec-8318-4e21-8e0a-510a49853a07,53867ee1-e97b-44c3-810b-cc0de8f49217,NaN,1.0,255,2021-07-09,7,2021,19,24307,67583
1,b7cf4711-f248-4a12-8bb6-d7a330faac5e,2021-07-06 23:46:09,c54bcaef-b3b9-4601-a5da-8519d33d3b81,89,1,f57143f5-672f-4c78-89f1-242f7a2d8a65,3f6de5e8-3b20-4237-89dc-ee4eff7625a3,08955352-31a8-44ac-8a5e-e13de23db4a2,df5f2a7b-ec52-4505-bd85-26a84438dee0,52eb5d34-a34b-4f29-9d83-e23d48d1e03b,NaN,1.0,89,2021-07-06,7,2021,19,31274,67070
2,0b16dd32-838f-4948-bd95-916d7b1a0a42,2021-02-01 15:06:29,abad439e-5bc0-4097-82e8-e87f0cb057ff,64,1,f57143f5-672f-4c78-89f1-242f7a2d8a65,68e916ca-f999-4e5a-9c34-f277f9bc2867,8c4f97b5-e3c1-4279-9829-ee56f920c12b,2b9626af-4919-45e4-873b-4a07a9749421,dbf9be47-c8e9-4fee-b92d-727f68afab96,NaN,1.0,64,2021-02-01,2,2021,14,27182,177935
3,a4998917-ae04-45e4-a5fe-4dc4a1fa4b35,2021-01-12 20:40:10,fc007164-7f6e-4833-9e36-e141c3fb4fe6,75,1,22f2e7c7-6679-41fd-9c1d-6e9562b51bce,025c6e08-592c-4f3d-8a9e-6d6b685fef33,4b52005f-878a-4b2b-95ee-d9e2d719046a,9c10f54e-63e5-4b8f-b720-4d95639db9bf,8e5145fd-9b3e-448d-b22e-a81991161656,NaN,8.0,75,2021-01-12,1,2021,13,40018,115195
4,a30dca13-d325-4a0c-a8fa-fb2ba762ad93,2021-07-05 00:03:59,f1fc2f7e-2aaa-484d-be31-8369b609cab0,75,2,451ffc74-a780-4ae0-adba-ff8caa8f4b91,025c6e08-592c-4f3d-8a9e-6d6b685fef33,4b52005f-878a-4b2b-95ee-d9e2d719046a,9c10f54e-63e5-4b8f-b720-4d95639db9bf,ba32d21e-9e8c-430b-b8f6-c11e0c34ffb7,NaN,8.0,150,2021-07-05,7,2021,19,38413,150503


In [4]:
df_train = pd.DataFrame()

for i in [17,19]:
    df = original_df[['user_id', 'dt', 'product', 'total_price', 'quantity', 'order_date_notime']]
    df = df[(df['dt']>=i) & (df['dt']<i+3)]
    df = df.drop(columns=['dt']).groupby(by=['user_id', 'product']).agg(
            total_price = ('total_price', 'sum'),
            quantity = ('quantity', 'sum'),
            times = ('product', 'count')).reset_index(inplace = False)
    df_train = pd.concat([df_train,df],axis=0, ignore_index=True)

,user_id,product,total_price,quantity,times
0,0,34773,98,2,1
1,0,16000,258,2,1
2,0,3365,358,2,1
3,0,37860,532,2,1
4,0,21736,39,1,1
...,...,...,...,...,...
5028960,207232,34304,1962,3,2
5028961,207232,18414,590,10,1
5028962,207232,23712,376,4,1
5028963,207232,38555,376,4,1


In [5]:
array_train = np.array([list(df_train.iloc[0,:])])
user_id = list(df_train['user_id'].unique())
for user in user_id[1:]:
#     user = user_id[1]
    user_df = df_train[df_train['user_id']==user].reset_index(drop = True)
    ls = [list(user_df.iloc[0,:])]
    array_train = np.append(array_train, ls, axis = 0)
    
array_train = pd.DataFrame(array_train, columns = ['user_id', 'product', 'total_price', 'quantity', 'times'])
array_train

,user_id,product,total_price,quantity,times
0,0,34773,98,2,1
1,1,27224,2358,4,4
2,3,25810,240,10,3
3,5,37002,315,3,2
4,6,32591,298,7,4
...,...,...,...,...,...
146131,207226,26068,567,7,1
146132,207227,5244,413,5,2
146133,207228,23308,506,7,3
146134,207231,11015,200,4,2


In [ ]:
train_y_df = original_df[original_df['dt'] >= 22]
train_y_df = train_y_df.sort_values(['user_id', 'order_date', 'quantity', 'total_price'], ascending = [True, False, False, False]).reset_index(drop = True)

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(train_result['product'].astype(int), train_result['product_predict'].astype(int), average='macro'))
print(f1_score(train_result['product'].astype(int), train_result['product_predict'].astype(int), average='micro'))
print(f1_score(train_result['product'].astype(int), train_result['product_predict'].astype(int), average='weighted'))

In [18]:
# original_df = pd.read_pickle('original_df_2.pkl')
df_train = pd.DataFrame()

for i in [17,19,21,22]: #[1,3,5,7,9,11,13,15,17,19,21,22]
    df = original_df[['user_id', 'dt', 'product', 'total_price', 'quantity', 'order_date_notime']]
    df = df[(df['dt']>=i) & (df['dt']<i+3)]
    df = df.drop(columns=['dt']).groupby(by=['user_id', 'product']).agg(
            total_price = ('total_price', 'sum'),
            quantity = ('quantity', 'sum'),
            times = ('product', 'count')).reset_index(inplace = False)
    df_train = pd.concat([df_train,df],axis=0, ignore_index=True)

## Training Data

In [ ]:
df_train = pd.DataFrame()

for i in [17,19,21]: #[1,3,5,7,9,11,13,15,17,19,21,22]
    df = original_df[['user_id', 'dt', 'product', 'total_price', 'quantity', 'order_date_notime']]
    df = df[(df['dt']>=i) & (df['dt']<i+3)]
    df = df.drop(columns=['dt']).groupby(by=['user_id', 'product']).agg(
            total_price = ('total_price', 'sum'),
            quantity = ('quantity', 'sum'),
            times = ('product', 'count')).reset_index(inplace = False)
    df_train = pd.concat([df_train,df],axis=0, ignore_index=True)

In [25]:
df_train = df_train.sort_values(['user_id', 'times', 'quantity', 'total_price'], ascending = [True, False, False, True]).reset_index(drop = True)
df_train

,user_id,product,total_price,quantity,times
0,0,34773,98,2,1
1,0,16000,258,2,1
2,0,16000,258,2,1
3,0,3365,358,2,1
4,0,3365,358,2,1
...,...,...,...,...,...
7236997,207232,23712,376,4,1
7236998,207232,38555,376,4,1
7236999,207232,6893,396,4,1
7237000,207232,34304,1962,3,1


In [78]:
array_train = np.array([list(df_train.iloc[0,:])])
array_train
user_id = list(df_train['user_id'].unique())
for user in user_id[1:]:
#     user = user_id[1]
    user_df = df_train[df_train['user_id']==user].reset_index(drop = True)
    ls = [list(user_df.iloc[0,:])]
    array_train = np.append(array_train, ls, axis = 0)
    array_train

In [80]:
array_train = pd.DataFrame(array_train, columns = ['user_id', 'product', 'total_price', 'quantity', 'times'])

In [83]:
# dt = 22
# df_test = pd.read_csv("./Training_Set_2/sum_at_{}_{}dt.csv".format(dt, dt+2))
# df_test = df_test.sort_values(['user_id', 'times', 'quantity', 'total_price'], ascending = [True, False, False, True]).reset_index(drop = True)
# df_test

,user_id,product,total_price,quantity,times
0,1,39891,55,1,1
1,1,32657,399,1,1
2,3,19453,490,5,4
3,3,27030,2235,15,3
4,3,35699,186,3,3
...,...,...,...,...,...
1226909,207232,35590,330,6,1
1226910,207232,14495,356,4,1
1226911,207232,35239,368,4,1
1226912,207232,6893,396,4,1


In [6]:
train_y_df = original_df[original_df['dt'] >= 22]
train_y_df = train_y_df.sort_values(['user_id', 'order_date', 'quantity', 'total_price'], ascending = [True, False, False, False]).reset_index(drop = True)
train_y_df

In [4]:
train_y = np.array([list(train_y_df.iloc[0,[0, 2, 17, 18]])])
user_id = list(train_y_df['user_id'].unique())
for user in user_id[1:]:
#     user = user_id[1]
    user_df = train_y_df[train_y_df['user_id']==user].reset_index(drop = True)
    ls = [list(user_df.iloc[0,[0, 2, 17, 18]])]
    train_y = np.append(train_y, ls, axis = 0)
    train_y
train_y = pd.DataFrame(train_y, columns = ['id', 'product_oroginal', 'product', 'user_id'])

In [11]:
train_y = train_y[['product_oroginal', 'product', 'user_id']]
train_y.rename(columns = {'product':'product_predict'}, inplace = True)
train_y

,product_oroginal,product_predict,user_id
0,cdc3d70e-3f93-4c5d-8ea0-d152e9b6589b,32657,1
1,b7113fca-0c14-44c4-945a-558b560cf154,28973,3
2,014c65f3-2286-49d0-9cfa-a3bbe77adc2e,218,5
3,4f6e3cc3-64cd-44ee-b5b2-18c1e1df7692,12541,6
4,2e20a1a2-2c5c-42e5-9001-61b900306f7f,7301,9
...,...,...,...
61252,3eaaa19e-fa45-485c-aa48-46668854b428,9980,207217
61253,bc454745-2c65-455d-a70c-57d72f6cae2f,29808,207220
61254,17840bb9-523b-4370-b97a-49324c1bc404,3707,207227
61255,3feb364a-8d30-4dda-8fd0-c3b7d3a93707,10162,207228


In [18]:
array_train['user_id'] = array_train['user_id'].astype('int')
train_y['user_id'] = train_y['user_id'].astype('int')
train_result = pd.merge(array_train[['user_id', 'product']], train_y[['user_id', 'product_predict']], on = ['user_id'], how = 'right')

In [28]:
from sklearn.metrics import f1_score
print(f1_score(train_result['product'].astype(int), train_result['product_predict'].astype(int), average='macro'))
print(f1_score(train_result['product'].astype(int), train_result['product_predict'].astype(int), average='micro'))
print(f1_score(train_result['product'].astype(int), train_result['product_predict'].astype(int), average='weighted'))

0.2832186537838645
0.40720244216987445
0.40754770613546104


## Testing Data

In [ ]:
df_test = pd.DataFrame()

for i in [17,19,21,22]: #[1,3,5,7,9,11,13,15,17,19,21,22]
    df = original_df[['user_id', 'dt', 'product', 'total_price', 'quantity', 'order_date_notime']]
    df = df[(df['dt']>=i) & (df['dt']<i+3)]
    df = df.drop(columns=['dt']).groupby(by=['user_id', 'product']).agg(
            total_price = ('total_price', 'sum'),
            quantity = ('quantity', 'sum'),
            times = ('product', 'count')).reset_index(inplace = False)
    df_test = pd.concat([df_test,df],axis=0, ignore_index=True)

In [86]:
array_test = np.array([list(df_test.iloc[0,:])])
user_id = list(df_test['user_id'].unique())
for user in user_id[1:]:
#     user = user_id[1]
    user_df = df_test[df_test['user_id']==user].reset_index(drop = True)
    ls = [list(user_df.iloc[0,:])]
    array_test = np.append(array_test, ls, axis = 0)
array_test = pd.DataFrame(array_test, columns = ['user_id', 'product', 'total_price', 'quantity', 'times'])
array_test

In [40]:
product_ls = original_df[['product', 'product_oroginal']].drop_duplicates()
product_ls = product_ls.drop_duplicates()

user_id_ls = original_df[['user_id', 'user_id_original']].drop_duplicates()
user_id_ls = user_id_ls.drop_duplicates()

,product,product_oroginal
0,24307,99d32476-f1eb-4af8-a7d9-28ebd8e1ce12
1,31274,c54bcaef-b3b9-4601-a5da-8519d33d3b81
2,27182,abad439e-5bc0-4097-82e8-e87f0cb057ff
3,40018,fc007164-7f6e-4833-9e36-e141c3fb4fe6
4,38413,f1fc2f7e-2aaa-484d-be31-8369b609cab0
...,...,...
17342916,40266,fd8ae9b5-7a6c-49b8-8bbe-3106577494e7
17343610,27824,afad2112-1795-481d-b813-a9f9a283ad0e
17343686,37213,ea8624e0-2d9b-46b7-b083-05b221de4506
17346683,31840,c8ba3b73-646d-42fd-916e-b2641b2cc416


In [45]:
predict = pd.merge(array_train, product_ls, on = ['product'], how = 'left')
predict = pd.merge(predict, user_id_ls, on = ['user_id'], how = 'left')

,user_id,product,total_price,quantity,times,product_oroginal,user_id_original
0,0,34773,98,2,1,db319208-4eed-4190-845c-2beb431f3436,00001f06-ae9a-4063-a22d-fd7754c89d74
1,1,27224,1764,3,3,ac000e4a-be9d-4a54-9d9c-6e3233bca1d2,00002857-64cb-4112-afb9-aa8e6eb79eb4
2,3,19453,686,7,5,7ae803ad-7c1c-4d58-ad48-c6c5f367a551,0001ae29-bf87-411e-afed-3545f835ac52
3,5,37002,315,3,2,e93d1fce-3cef-42c4-86ac-491badaf9c8c,00021382-790a-49a5-a9bd-88b0c42aa598
4,6,32591,298,7,4,cd64ff5c-7f67-42f5-9335-29b45acd46d1,000265b1-95ad-487d-824f-30752b7486ff
...,...,...,...,...,...,...,...
148578,207226,26068,567,7,1,a4aad0f0-f32f-420a-b09d-f47c09084c88,fffea78c-4972-4848-bad1-9bbc6ce52b2f
148579,207227,5244,413,5,2,20af9570-574c-4f39-a596-3a2910806669,fffeb89d-85fb-49ce-a414-efffe860ce29
148580,207228,28908,33,3,3,b6a4357a-2966-4678-9d0f-adcd862c21ba,ffff742d-a685-4161-bc54-4b4dac33ec39
148581,207231,11015,200,4,2,4586fe5d-11c5-42e9-8cd7-866db2f58e88,fffff859-d3e9-4e6f-840a-5112618a3b77


In [22]:
original_df_product = original_df.groupby('product_oroginal').agg(
    total_price = ('total_price', 'sum'),
    quantity = ('quantity', 'sum'),
    times = ('product', 'count')).reset_index(inplace = False)

original_df_product = original_df_product.sort_values(['times', 'quantity', 'total_price'], ascending = [False, False, False]).reset_index(drop = True)
first_product = original_df_product['product_oroginal'][0]


'7f7a7405-c43b-4cab-ac38-dc279aa12648'

In [41]:
user_id_original = original_df['user_id_original'].unique()
user_id_original = pd.DataFrame({'user_id_original': user_id_original})

In [50]:
result = pd.merge(user_id_original, predict[['user_id_original', 'product_oroginal']], on = ['user_id_original'], how = 'left')
result['product_oroginal'] = result['product_oroginal'].fillna(first_product)
result.rename(columns = {'user_id_original':'user_id', 'product_oroginal':'product'}, inplace = True)
result

,user_id,product
0,53867ee1-e97b-44c3-810b-cc0de8f49217,b976ca58-d70b-42e5-ab19-b65935330eef
1,52eb5d34-a34b-4f29-9d83-e23d48d1e03b,b01e1c80-e6c3-4bd9-ac6d-d2a3236d750a
2,dbf9be47-c8e9-4fee-b92d-727f68afab96,94dfbaec-94d2-466c-980d-8e32b87b05f8
3,8e5145fd-9b3e-448d-b22e-a81991161656,7f7a7405-c43b-4cab-ac38-dc279aa12648
4,ba32d21e-9e8c-430b-b8f6-c11e0c34ffb7,fe9f62aa-758c-4bfb-80bc-d01951399d6b
...,...,...
207228,118b33f5-5e19-4a41-92b8-f70c549d80ee,460d4ace-de99-4a9a-8646-0571680a8b5e
207229,f9763964-3042-4777-95bd-bf48a037d32a,7f7a7405-c43b-4cab-ac38-dc279aa12648
207230,00255826-5f4f-4e4d-ba96-cd672051e72a,7f7a7405-c43b-4cab-ac38-dc279aa12648
207231,a902a465-7610-4ce0-a430-9156d1c716e2,7f7a7405-c43b-4cab-ac38-dc279aa12648


In [51]:
result.to_csv("predict_result.csv", index=False)